In [ ]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
from math import pi, cos, sin, tan

In [ ]:
def pad_ones(x, y, z):
    return np.array([x, y, z, np.ones_like(x)])

In [ ]:
def project_vector(x, y, z, matrix):
    vec = np.dot(matrix, pad_ones(x, y, z))

    return vec[0]/vec[3], vec[1]/vec[3], vec[2]/vec[3]

In [ ]:
def normalize(vec):
    return vec / np.linalg.norm(vec)

In [ ]:
def get_look_at_matrix(eye, center, up):
    forward = normalize(center - eye)
    side = normalize(np.cross(forward, up))

    # Compute the real up
    up = normalize(np.cross(side, forward))

    # Compute translation factors
    tx = - side[0] * eye[0] - side[1] * eye[1] - side[2] * eye[2] + 1
    ty = - up[0] * eye[0] - up[1] * eye[1] - up[2] * eye[2] + 1
    tz = forward[0] * eye[0] + forward[1] * eye[1] + forward[2] * eye[2] + 1

    return np.array([
        [    side[0],     side[1],     side[2], tx],
        [      up[0],       up[1],       up[2], ty],
        [-forward[0], -forward[1], -forward[2], tz],
        [          0,           0,           0,  1]
    ])

In [ ]:
def get_perspective_matrix(fovy, aspect, near, far):
    f = 1. / tan(fovy * pi / 360.)

    return np.array([
        [f/aspect, 0,                           0,                           0],
        [       0, f,                           0,                           0],
        [       0, 0,   (near + far)/(near - far), 2 * near * far/(near - far)],
        [       0, 0,                          -1,                           0]
    ])

In [ ]:
def get_projection(elev, azim, radius, aspect):
    relev, razim = np.pi * elev/180, np.pi * azim/180

    center = np.array([0, 0, 0])

    xp = cos(razim) * cos(relev) * radius
    yp = sin(razim) * cos(relev) * radius
    zp = sin(relev) * radius
    eye = - np.array((xp, yp, zp))

    if abs(relev) > pi / 2.:
        up = np.array((0, 0, -1))
    else:
        up = np.array((0, 0, 1))

    view_matrix = get_look_at_matrix(eye, center, up)
    projection_matrix = get_perspective_matrix(70, aspect, 0.5, 2 * radius)
    return np.dot(projection_matrix, view_matrix)

In [ ]:
class Plot3d(Canvas):
    def __init__(self):
        super(Plot3d, self).__init__(size=(500, 500))
        
        self.width = 500
        self.height = 500

        self.dragging = False
        self.n = 200
        self.x = np.random.rand(self.n) - 0.5
        self.y = np.random.rand(self.n) - 0.5
        self.z = np.random.rand(self.n) - 0.5

        self.radius = 5
        self.dx = 0
        self.dy = 0
        self.matrix = get_projection(self.dy, self.dx, self.radius, self.width / self.height)
        self.x2, self.y2, _ = project_vector(self.x, self.y, self.z, self.matrix)
        self.draw()

        self.on_mouse_down(self.mouse_down_handler)
        self.on_mouse_move(self.mouse_move_handler)
        self.on_mouse_up(self.mouse_up_handler)
        self.on_mouse_out(self.mouse_out_handler)

    def draw(self):
        x = self.x2 * self.width + self.width / 2
        y = self.y2 * self.height + self.height / 2
        with hold_canvas(self):
            self.clear()
            self.fill_circles(x, y, 3)

    def mouse_down_handler(self, pixel_x, pixel_y):
        self.dragging = True
        self.x_mouse = pixel_x
        self.y_mouse = pixel_y

    def mouse_move_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dx_new = self.dx + pixel_x - self.x_mouse
            self.dy_new = self.dy + pixel_y - self.y_mouse

            self.matrix = get_projection(self.dy_new, self.dx_new, self.radius, self.width / self.height)
            self.x2, self.y2, _ = project_vector(self.x, self.y, self.z, self.matrix)
            self.draw()
    
    def mouse_up_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new
    
    def mouse_out_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new

In [ ]:
p = Plot3d()
p